In [ ]:
import random
import json
import time

class Item:
    def __init__(self, name, effect):
        self.name = name
        self.effect = effect

class Weapon(Item):
    def __init__(self, name, attack_bonus):
        super().__init__(name, effect=f"Attack Bonus: {attack_bonus}")
        self.attack_bonus = attack_bonus

class Consumable(Item):
    def __init__(self, name, heal_amount):
        super().__init__(name, effect=f"Heals {heal_amount} HP")
        self.heal_amount = heal_amount

class Player:
    def __init__(self, name, health, attack, defense, weapon):
        self.name = name
        self.level = 1
        self.experience = 0
        self.health = health
        self.max_health = 30
        self.attack = attack
        self.defense = defense
        self.weapon = weapon
        self.inventory = self.load_inventory()
    
    def load_inventory(self):
        try:
            with open("inventory.json", "r") as file:
                return json.load(file)
        except FileNotFoundError:
            return {
                "swords": {}, "food": {}, "daggers": {}, "wands": {}, "staffs": {}, "potions": {}
            }
    
    def save_inventory(self):
        with open("inventory.json", "w") as file:
            json.dump(self.inventory, file, indent=4)

    def save_player(self):
        player_data = {
            "name": self.name,
            "level": self.level,
            "experience": self.experience,
            "health": self.health,
            "max_health": self.max_health,
            "attack": self.attack,
            "defense": self.defense,
            "weapon": self.weapon,
        }
        with open("player_data.json", "w") as file:
            json.dump(player_data, file, indent=4)
        print(f"{self.name}'s data saved!")

    @classmethod
    def load_player(cls):
        try:
            with open("player_data.json", "r") as file:
                player_data = json.load(file)
                player = cls(
                    player_data["name"],
                    player_data["health"],
                    player_data["attack"],
                    player_data["defense"],
                    player_data["weapon"]
                )
                player.level = player_data["level"]
                player.experience = player_data["experience"]
                player.max_health = player_data["max_health"]
                return player
        except FileNotFoundError:
            print("No saved player data found!")
            return None
    
    def equip_weapon(self, weapon_name):
        for category in ["swords", "daggers", "wands", "staffs"]:
            if weapon_name in self.inventory[category]:
                self.weapon = weapon_name
                print(f"{self.name} equips {weapon_name}!")
                return
        print("Weapon not found in inventory!")
    
    def show_inventory(self):
        print("Inventory:")
        for category, items in self.inventory.items():
            print(f"{category.capitalize()}: {', '.join(items.keys()) if items else 'None'}")
        input("Press Enter to go back.")
    
    def use_item(self):
        while True:
            self.show_inventory()
            item_name = input("Enter the item name to use or type 'back' to return: ")
            if item_name.lower() == 'back':
                return False  # Returns False to indicate no turn taken
            for category in ["food", "potions"]:
                if item_name in self.inventory[category]:
                    self.health = min(self.max_health, self.health + 20)
                    print(f"{self.name} uses {item_name} and restores health! Current health: {self.health}")
                    del self.inventory[category][item_name]
                    self.save_inventory()
                    return True  # Returns True to indicate a turn was taken
            print("Item not found in inventory!")
    
    def take_damage(self, damage):
        damage_taken = max(0, damage - self.defense)
        self.health -= damage_taken
        print(f"{self.name} takes {damage_taken} damage! Remaining health: {self.health}")
        if self.health <= 0:
            print(f"{self.name} has been defeated!")
    
    def gain_experience(self, amount):
        if self.level == 99:
            print(f"{self.name} gains {amount} EXP but remains at level 99.")
            return
        self.experience += amount
        print(f"{self.name} gains {amount} EXP! Current EXP: {self.experience}")
        self.check_level_up()
    
    def check_level_up(self):
        required_exp = self.level * 50
        while self.experience >= required_exp and self.level < 99:
            self.experience -= required_exp
            self.level += 1
            self.max_health = int(30 + (self.level - 1) * (550 / 98))
            self.health = self.max_health
            print(f"{self.name} leveled up to Level {self.level}! Max Health is now {self.max_health}.")
            required_exp = self.level * 50
    
    def attack_enemy(self, enemy):
        damage = self.attack + random.randint(1, 5)
        print(f"{self.name} attacks {enemy.name} with {self.weapon} for {damage} damage! {enemy.name} has {enemy.health - damage} HP left.")
        enemy.take_damage(damage)
    
class Warrior(Player):
    def __init__(self, name):
        super().__init__(name, health=30, attack=15, defense=10, weapon="Sword")

class Mage(Player):
    def __init__(self, name):
        super().__init__(name, health=30, attack=20, defense=5, weapon="Wand")
    
    def heal(self, ally):
        if self.level >= 5:
            heal_amount = 5
            ally.health = min(ally.max_health, ally.health + heal_amount)
            print(f"{self.name} heals {ally.name} for {heal_amount} health! {ally.name} now has {ally.health} HP.")
        else:
            print(f"{self.name} has not unlocked healing yet! (Requires level 5)")
    
class Priest(Player):
    def __init__(self, name):
        super().__init__(name, health=30, attack=8, defense=7, weapon="Staff")
    
    def heal(self, ally):
        if self.level >= 5:
            heal_amount = 5
            ally.health = min(ally.max_health, ally.health + heal_amount)
            print(f"{self.name} heals {ally.name} for {heal_amount} health! {ally.name} now has {ally.health} HP.")
        else:
            print(f"{self.name} has not unlocked healing yet! (Requires level 5)")
    
class Thief(Player):
    def __init__(self, name):
        super().__init__(name, health=30, attack=12, defense=8, weapon="Daggers")
    
class Slime(Player):
    def __init__(self):
        super().__init__("Slime", health=60, attack=5, defense=2, weapon="Slime Body")
    
    def attack_enemy(self, player):
        damage = self.attack + random.randint(1, 3)
        print(f"{self.name} attacks {player.name} for {damage} damage!")
        player.take_damage(damage)

def battle(players, enemy):
    print("A Slime draws near!")
    for player in players:
        print(f"{player.name}: {player.health}/{player.max_health} HP")
        # 4 lines above tell you how much HP each of your party has 
    while enemy.health > 0 and any(player.health > 0 for player in players):
        for player in players:
            if player.health > 0:
                while True:
                    action = input(f"{player.name}, choose your action (attack, heal, use item, inventory): ").strip().lower()
                    if action in ["attack", "heal", "use item", "inventory"]:
                        break
                    print("Invalid choice, please try again.")
                if action == "attack":
                    player.attack_enemy(enemy)
                elif action == "heal" and isinstance(player, (Priest, Mage)):
                    target = random.choice(players)
                    player.heal(target)
                elif action == "use item":
                    if not player.use_item():
                        continue
                elif action == "inventory":
                    player.show_inventory()
                    continue

                if enemy.health > 0:
                    enemy.attack_enemy(player)
                if enemy.health <= 0:
                    print("The Slime is defeated!")
                    player.gain_experience(50)
                    return

def main():
    print("Welcome, adventurer! Loading your saved game if available...")
    
    # Try to load the saved player
    player = Player.load_player()
    
    if not player:
        # If no saved player is found, ask for class selection
        print("No saved player found. Please choose your class.")
        hero_classes = {"warrior": Warrior, "mage": Mage, "priest": Priest, "thief": Thief}
        
        while True:
            hero_type = input("Choose your class (Warrior, Mage, Priest, Thief): ").strip().lower()
            if hero_type in hero_classes:
                break
            print("Invalid choice, please try again.")
        
        hero_name = input("Enter your hero's name: ")
        player = hero_classes[hero_type](hero_name)  # Create new player based on the class chosen
    
    else:
        # If a saved player is loaded, prevent class change and just inform the user of their class
        print(f"Welcome back, {player.name}! You are playing as a {player.__class__.__name__.capitalize()}.")
    
    print("Loading...")
    time.sleep(3)
    print("Your quest begins now!")
    battle([player], Slime())
    
    # Save player data after battle
    player.save_player()

if __name__ == "__main__":
    main()

Welcome, adventurer! Loading your saved game if available...
Welcome back, Anthony! You are playing as a Player.
Loading...
Your quest begins now!
A Slime draws near!
Anthony: 41/41 HP
Anthony attacks Slime with Sword for 20 damage! Slime has 40 HP left.
Slime takes 18 damage! Remaining health: 42
Slime attacks Anthony for 6 damage!
Anthony takes 0 damage! Remaining health: 41
Anthony attacks Slime with Sword for 17 damage! Slime has 25 HP left.
Slime takes 15 damage! Remaining health: 27
Slime attacks Anthony for 8 damage!
Anthony takes 0 damage! Remaining health: 41
Anthony attacks Slime with Sword for 19 damage! Slime has 8 HP left.
Slime takes 17 damage! Remaining health: 10
Slime attacks Anthony for 8 damage!
Anthony takes 0 damage! Remaining health: 41
Anthony attacks Slime with Sword for 19 damage! Slime has -9 HP left.
Slime takes 17 damage! Remaining health: -7
Slime has been defeated!
The Slime is defeated!
Anthony gains 50 EXP! Current EXP: 150
Anthony leveled up to Level 4!